# Chapter 28: Argparse Fundamentals

This notebook covers the `argparse` module for building command-line interfaces in Python. We explore `ArgumentParser`, positional and optional arguments, type conversion, choices, defaults, and `nargs`.

## Key Concepts
- **ArgumentParser**: The main entry point for defining CLI arguments
- **Positional arguments**: Required arguments identified by position
- **Optional arguments**: Flag-style arguments with `--` prefix
- **Type conversion**: Automatic conversion of string inputs to desired types
- **Choices and defaults**: Restricting and pre-filling argument values

## Section 1: Creating an ArgumentParser

`ArgumentParser` is the central class in `argparse`. It holds all the information needed to parse the command line into Python data types.

In [ ]:
import argparse

# Create a basic parser with a description
parser: argparse.ArgumentParser = argparse.ArgumentParser(
    description="A sample command-line tool",
    prog="mytool",  # Override the program name
)

print(f"Parser type: {type(parser).__name__}")
print(f"Program name: {parser.prog}")
print(f"Description: {parser.description}")

In [ ]:
# The parser generates help text automatically
parser = argparse.ArgumentParser(
    prog="greet",
    description="Greet someone on the command line",
    epilog="Example: greet Alice --loud",
)

# print_help() shows what --help would display
parser.print_help()

## Section 2: Positional Arguments

Positional arguments are required by default and identified by their position on the command line rather than by a flag.

In [ ]:
# Positional arguments are required
parser = argparse.ArgumentParser(prog="greet")
parser.add_argument("name")  # A simple positional argument

# parse_args takes a list of strings (simulating sys.argv[1:])
args: argparse.Namespace = parser.parse_args(["Alice"])

print(f"Parsed name: {args.name}")
print(f"Namespace type: {type(args).__name__}")
print(f"As dict: {vars(args)}")

In [ ]:
# Multiple positional arguments are parsed in order
parser = argparse.ArgumentParser(prog="copy")
parser.add_argument("source")
parser.add_argument("destination")

args = parser.parse_args(["input.txt", "output.txt"])

print(f"Source: {args.source}")
print(f"Destination: {args.destination}")

## Section 3: Optional Arguments

Optional arguments use the `--` prefix (long form) or `-` prefix (short form). They are not required unless you set `required=True`.

In [ ]:
# Optional arguments with -- prefix
parser = argparse.ArgumentParser(prog="server")
parser.add_argument("--verbose", action="store_true")
parser.add_argument("--count", type=int, default=1)

# Providing both optional arguments
args = parser.parse_args(["--verbose", "--count", "5"])
print(f"verbose: {args.verbose} (type: {type(args.verbose).__name__})")
print(f"count: {args.count} (type: {type(args.count).__name__})")

# Omitting optional arguments uses defaults
args_default = parser.parse_args([])
print(f"\nDefaults -> verbose: {args_default.verbose}, count: {args_default.count}")

In [ ]:
# Short and long flags together
parser = argparse.ArgumentParser(prog="tool")
parser.add_argument("-v", "--verbose", action="store_true")
parser.add_argument("-o", "--output", type=str, default="stdout")

# Using short flags
args = parser.parse_args(["-v", "-o", "results.txt"])
print(f"verbose: {args.verbose}")
print(f"output: {args.output}")

# Using long flags produces the same result
args_long = parser.parse_args(["--verbose", "--output", "results.txt"])
print(f"\nSame result with long flags: {vars(args) == vars(args_long)}")

## Section 4: Type Conversion

By default, all arguments are parsed as strings. The `type` parameter automatically converts the input to the desired Python type.

In [ ]:
# Built-in type conversion
parser = argparse.ArgumentParser(prog="calc")
parser.add_argument("value", type=int)
parser.add_argument("--factor", type=float, default=1.0)

args = parser.parse_args(["42", "--factor", "2.5"])

result: float = args.value * args.factor
print(f"value: {args.value} (type: {type(args.value).__name__})")
print(f"factor: {args.factor} (type: {type(args.factor).__name__})")
print(f"result: {result}")

In [ ]:
import pathlib

# Custom type functions for validation
def positive_int(value: str) -> int:
    """Convert string to positive integer, raising on invalid input."""
    ivalue: int = int(value)
    if ivalue <= 0:
        raise argparse.ArgumentTypeError(f"{value} is not a positive integer")
    return ivalue

parser = argparse.ArgumentParser(prog="worker")
parser.add_argument("--threads", type=positive_int, default=4)
parser.add_argument("--config", type=pathlib.Path, default=pathlib.Path("config.toml"))

args = parser.parse_args(["--threads", "8", "--config", "/etc/app/config.toml"])
print(f"threads: {args.threads} (type: {type(args.threads).__name__})")
print(f"config: {args.config} (type: {type(args.config).__name__})")

## Section 5: Choices and Defaults

The `choices` parameter restricts an argument to a set of allowed values. The `default` parameter sets a fallback when the argument is omitted.

In [ ]:
# choices restricts accepted values
parser = argparse.ArgumentParser(prog="logger")
parser.add_argument(
    "--level",
    choices=["debug", "info", "warning", "error", "critical"],
    default="info",
)

args = parser.parse_args(["--level", "debug"])
print(f"Log level: {args.level}")

# Default value when argument is omitted
args_default = parser.parse_args([])
print(f"Default level: {args_default.level}")

In [ ]:
# Demonstrating what happens with invalid choices
parser = argparse.ArgumentParser(prog="format")
parser.add_argument("--output-format", choices=["json", "csv", "xml"])

# Valid choice
args = parser.parse_args(["--output-format", "json"])
print(f"Format: {args.output_format}")

# Invalid choice raises an error
try:
    parser.parse_args(["--output-format", "yaml"])
except SystemExit:
    print("Invalid choice 'yaml' - argparse rejects it and exits")

## Section 6: Nargs — Multiple Values

The `nargs` parameter controls how many values an argument consumes:
- `nargs='+'` — one or more values (returns a list)
- `nargs='*'` — zero or more values (returns a list)
- `nargs='?'` — zero or one value
- `nargs=N` — exactly N values (returns a list)

In [ ]:
# nargs='+' requires one or more values
parser = argparse.ArgumentParser(prog="process")
parser.add_argument("files", nargs="+")

args = parser.parse_args(["a.txt", "b.txt", "c.txt"])
print(f"Files: {args.files}")
print(f"Type: {type(args.files).__name__}")
print(f"Count: {len(args.files)}")

In [ ]:
# nargs='*' accepts zero or more values
parser = argparse.ArgumentParser(prog="collect")
parser.add_argument("--tags", nargs="*", default=[])
parser.add_argument("--ports", nargs=3, type=int)  # Exactly 3 values

args = parser.parse_args(["--tags", "dev", "test", "--ports", "80", "443", "8080"])
print(f"Tags: {args.tags}")
print(f"Ports: {args.ports}")

# Zero tags is valid with nargs='*'
args_empty = parser.parse_args(["--tags", "--ports", "80", "443", "8080"])
print(f"\nEmpty tags: {args_empty.tags}")

## Section 7: Actions

The `action` parameter controls how arguments are processed:
- `"store"` — default, stores the value
- `"store_true"` / `"store_false"` — boolean flags
- `"count"` — counts occurrences (e.g., `-vvv`)
- `"append"` — collects repeated arguments into a list

In [ ]:
# Demonstrating various actions
parser = argparse.ArgumentParser(prog="demo")

# store_true / store_false for boolean flags
parser.add_argument("--debug", action="store_true")
parser.add_argument("--no-cache", action="store_true")

# count tracks how many times a flag appears
parser.add_argument("-v", "--verbose", action="count", default=0)

# append collects repeated flags into a list
parser.add_argument("--include", action="append", default=[])

args = parser.parse_args([
    "--debug",
    "-vvv",  # Three v's = verbosity 3
    "--include", "src",
    "--include", "tests",
])

print(f"debug: {args.debug}")
print(f"no_cache: {args.no_cache}")
print(f"verbosity: {args.verbose}")
print(f"includes: {args.include}")

## Section 8: Help Text and Metavar

Good CLI tools provide clear help text. The `help` parameter adds descriptions, and `metavar` customizes the placeholder name in usage messages.

In [ ]:
# Well-documented argument parser
parser = argparse.ArgumentParser(
    prog="deploy",
    description="Deploy an application to the target environment",
)
parser.add_argument(
    "app_name",
    help="Name of the application to deploy",
    metavar="APP",
)
parser.add_argument(
    "--env",
    choices=["dev", "staging", "prod"],
    default="dev",
    help="Target environment (default: %(default)s)",
)
parser.add_argument(
    "--replicas",
    type=int,
    default=1,
    metavar="N",
    help="Number of replicas to run (default: %(default)s)",
)
parser.add_argument(
    "--dry-run",
    action="store_true",
    help="Show what would be done without executing",
)

parser.print_help()

In [ ]:
# Parsing with the documented parser
args = parser.parse_args(["myapp", "--env", "staging", "--replicas", "3", "--dry-run"])

print(f"App: {args.app_name}")
print(f"Environment: {args.env}")
print(f"Replicas: {args.replicas}")
print(f"Dry run: {args.dry_run}")

# Namespace can be accessed as a dictionary too
config: dict[str, object] = vars(args)
print(f"\nFull config: {config}")

## Section 9: Putting It All Together

A realistic example combining positional arguments, optional flags, types, choices, and defaults.

In [ ]:
def build_parser() -> argparse.ArgumentParser:
    """Build a complete argument parser for a file search tool."""
    parser = argparse.ArgumentParser(
        prog="filesearch",
        description="Search for patterns in files",
    )

    # Positional: the pattern to search for
    parser.add_argument("pattern", help="Regex pattern to search for")

    # Positional: one or more files
    parser.add_argument("files", nargs="+", help="Files to search")

    # Optional flags
    parser.add_argument("-i", "--ignore-case", action="store_true",
                        help="Case-insensitive matching")
    parser.add_argument("-n", "--line-numbers", action="store_true",
                        help="Show line numbers")
    parser.add_argument("--max-results", type=positive_int, default=100,
                        metavar="N", help="Maximum results (default: %(default)s)")
    parser.add_argument("--format", choices=["text", "json", "csv"],
                        default="text", help="Output format (default: %(default)s)")

    return parser


search_parser: argparse.ArgumentParser = build_parser()
search_parser.print_help()

In [ ]:
# Simulate command-line invocation
args = search_parser.parse_args([
    "TODO",
    "main.py", "utils.py", "config.py",
    "--ignore-case",
    "--line-numbers",
    "--max-results", "50",
    "--format", "json",
])

print(f"Pattern: {args.pattern!r}")
print(f"Files: {args.files}")
print(f"Ignore case: {args.ignore_case}")
print(f"Line numbers: {args.line_numbers}")
print(f"Max results: {args.max_results}")
print(f"Format: {args.format}")

## Summary

### ArgumentParser Basics
- **`ArgumentParser()`** creates a parser with optional `prog`, `description`, and `epilog`
- **`add_argument()`** defines arguments with their type, default, help, and constraints
- **`parse_args()`** returns a `Namespace` object with parsed values

### Argument Types
- **Positional**: Required by default, identified by order
- **Optional**: Prefixed with `--` (long) or `-` (short), not required by default

### Key Parameters
- **`type`**: Converts input strings to the desired Python type
- **`choices`**: Restricts values to a predefined set
- **`default`**: Fallback value when argument is omitted
- **`nargs`**: Controls how many values an argument accepts (`+`, `*`, `?`, or `N`)
- **`action`**: Controls processing (`store_true`, `count`, `append`)
- **`help`** and **`metavar`**: Document the argument for users